## THỰC HÀNH MHNC LAB 05

### Nguyễn Minh Hoàng 18110095

In [1]:
import pandas as pd
import numpy as np
import nltk
import io

from tensorflow.keras.layers import ( 
                              Input, Embedding, LSTM, Bidirectional, 
                               Dense, Dropout, BatchNormalization,GRU,
                               GlobalAveragePooling1D)
from tensorflow.keras.models import Model


In [2]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, GRU, GlobalAveragePooling1D
from tensorflow.keras.models import Model

In [3]:
#đọc data
df = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/AML/main/lab-04/spam_detection.csv")
df.head()

,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0


In [4]:
df.shape

(5574, 3)

In [5]:
nltk.download('punkt')
texts = df["Text"].to_list()
texts = [text.lower() for text in texts ]           # chuyển các đoạn text thành chữ thường (word embedding chỉ cho chữ thường)
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    # tách câu thành một list các từ

print(tokenized_texts[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['go', 'until', 'jurong', 'point', ',', 'crazy..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']


Để Vectorize các câu ta sử dụng một bộ trọng số có tên là Glove do Stanford phát triển. bộ weights này sẽ biến mỗi chữ thành 1 vector có số nhiều là 50, 100, 200, hoặc 300. Số chiều càng lớn thì vector sẽ càng biểu diễn được ngữ nghĩa của một từ 

In [6]:
#Tải bộ weights Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-11-20 13:42:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-11-20 13:42:07--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-11-20 13:42:07--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [7]:
#Giải Nén
! unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.200d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace glove.6B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glove.6B.300d.txt       


In [8]:
## không cần hiểu đống này lắm đâu
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix) 

In [9]:
#vocab là bộ từ điển các từ có trong Glove
#matrix chứa các vector biểu diễn các từ 
##Ta chọn bộ weighs có số chiều là 100
vocab, matrix = load_word_embeddings("glove.6B.100d.txt")

In [10]:
len(vocab)

400000

In [11]:
#in thử shape của matrix
##Ta có thể thấy bộ từ điển này có 400000, mỗi từ là 1 vecotr có 100 chiều 
matrix.shape

(400000, 100)

In [12]:
## Gán các mã
__PADDED_INDEX__ = 0    # mã dùng cho các vị trí chỉ có tính nối dài cho cùng kích thước
__UNKNOWN_WORD__ = 1    # mã cho những từ không có trong embedding

In [13]:
# Tạo một dictionary, có nhiệm vụ là một ánh xạ từ ảnh sang mã số, mã số được bắt đầu từ 2 vì số 0 và 1 được dành cho trường hợp đặc biệt
word_to_index = {word: index+2 for index, word in enumerate(vocab)}

In [14]:

# Do do mã số được bắt đầu từ 2, nên cần thêm 2 vector vào đàu ma trận
embedding_matrix = np.pad(matrix, ((2,0),(0,0)), mode='constant', constant_values =0.0)
print(embedding_matrix)

[[ 0.        0.        0.       ...  0.        0.        0.      ]
 [ 0.        0.        0.       ...  0.        0.        0.      ]
 [-0.038194 -0.24487   0.72812  ... -0.1459    0.8278    0.27062 ]
 ...
 [ 0.36088  -0.16919  -0.32704  ...  0.27139  -0.29188   0.16109 ]
 [-0.10461  -0.5047   -0.49331  ...  0.42527  -0.5125   -0.17054 ]
 [ 0.28365  -0.6263   -0.44351  ...  0.43678  -0.82607  -0.15701 ]]


In [15]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

In [16]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [17]:
## Chia data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state =0)



####1. Tìm hiểu về cách dùng của GRU và chạy theo 2 cách hướng dẫn ở trên 

### LSTM

In [18]:

inputs = Input(shape=(None,))                   ## None biểu thị kích thước không xác định của câu

embed = Embedding(input_dim=embedding_matrix.shape[0],   ## Khai báo kích thước của vocab
                  output_dim=embedding_matrix.shape[1],   ## Khai báo kích thước của embedding
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable=False,                       ## Không cần thiết train embedding
                  mask_zero=True)(inputs)                 ## zero_mask: những vị trí có giá trị 0 không được tính toán, vì đó là giá trị thêm vào cho đủ độ dài mà thôi
                                                        ##  (__PADDED_INDEX__ gán bằng 0)

lstm = LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng
lstm = Dropout(0.2)(lstm)
lstm = BatchNormalization()(lstm)
outputs = Dense(units=2, 
              activation='softmax')(lstm)
model_lstm1 = Model(inputs=inputs,
              outputs=outputs)



model_lstm1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         40000200  
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                             

In [19]:
list_times = []

In [20]:
import timeit
start = timeit.default_timer() 

model_lstm1.compile(
                optimizer = tf.keras.optimizers.RMSprop(learning_rate= 1e-3, name='RMSprop'),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


history = model_lstm1.fit(X_train, y_train,
                    batch_size= 100,
                    epochs= 10) 
     
stop = timeit.default_timer()
print('Time: ', stop - start) 
list_times.append(stop-start)

Epoch 1/10
45/45 [==============================] - 8s 33ms/step - loss: 0.4017 - accuracy: 0.8394
Epoch 2/10
45/45 [==============================] - 1s 33ms/step - loss: 0.1295 - accuracy: 0.9641
Epoch 3/10
45/45 [==============================] - 1s 32ms/step - loss: 0.1046 - accuracy: 0.9762
Epoch 4/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0704 - accuracy: 0.9798
Epoch 5/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0664 - accuracy: 0.9794
Epoch 6/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0442 - accuracy: 0.9865
Epoch 7/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0433 - accuracy: 0.9856
Epoch 8/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0402 - accuracy: 0.9879
Epoch 9/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0253 - accuracy: 0.9919
Epoch 10/10
45/45 [==============================] - 1s 31ms/step - loss: 0.0291 - accuracy: 0.9919
Time:  25

In [36]:
performance = []

In [38]:
result = model_lstm1.evaluate(X_valid,y_valid)
print("loss và accuracy trên tập test là {} {}".format(result[0],result[1]))

35/35 [==============================] - 0s 10ms/step - loss: 0.0546 - accuracy: 0.9821
loss và accuracy trên tập test là 0.054580964148044586 0.9820627570152283


In [39]:
performance.append(result[1])

In [22]:
inputs = Input(shape=(None,))                   
embed = Embedding(input_dim=embedding_matrix.shape[0],   
                output_dim=embedding_matrix.shape[1],   
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  
                  trainable=False,                      
                mask_zero=True)(inputs)                
                                                        
lstm = LSTM(units=100,                         
            return_sequences=True,
            return_state=True)(embed)     
lstm = GlobalAveragePooling1D()(lstm[0])
lstm = Dropout(0.2)(lstm)
lstm = BatchNormalization()(lstm)
outputs = Dense(units=2, activation='softmax')(lstm)

model_lstm2 = Model(inputs=inputs,
              outputs=outputs)


model_lstm2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 100)         40000200  
                                                                 
 lstm_1 (LSTM)               [(None, None, 100),       80400     
                              (None, 100),                       
                              (None, 100)]                       
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                           

In [23]:
start = timeit.default_timer() 
model_lstm2.compile(
                optimizer = tf.keras.optimizers.RMSprop(learning_rate= 1e-3, name='RMSprop'),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


history = model_lstm2.fit(X_train, y_train,
                    batch_size= 100,
                    epochs= 10) 

stop = timeit.default_timer()
print('Time: ', stop - start) 
list_times.append(stop-start)     


Epoch 1/10
45/45 [==============================] - 7s 32ms/step - loss: 0.4294 - accuracy: 0.8246
Epoch 2/10
45/45 [==============================] - 1s 33ms/step - loss: 0.1516 - accuracy: 0.9641
Epoch 3/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0956 - accuracy: 0.9738
Epoch 4/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0821 - accuracy: 0.9771
Epoch 5/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0653 - accuracy: 0.9782
Epoch 6/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0602 - accuracy: 0.9814
Epoch 7/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0471 - accuracy: 0.9850
Epoch 8/10
45/45 [==============================] - 1s 33ms/step - loss: 0.0439 - accuracy: 0.9870
Epoch 9/10
45/45 [==============================] - 1s 33ms/step - loss: 0.0354 - accuracy: 0.9906
Epoch 10/10
45/45 [==============================] - 1s 32ms/step - loss: 0.0343 - accuracy: 0.9906
Time:  25

In [41]:

result = model_lstm2.evaluate(X_valid,y_valid)
print("loss và accuracy trên tập val là {} {}".format(result[0],result[1]))

35/35 [==============================] - 0s 10ms/step - loss: 0.0636 - accuracy: 0.9776
loss và accuracy trên tập val là 0.06356462091207504 0.9775784611701965


In [42]:
performance.append(result[1])

### GRU

In [25]:
list_times1 = []

In [26]:
inputs = Input(shape=(None,))          
embed = Embedding(input_dim=embedding_matrix.shape[0],   
                  output_dim=embedding_matrix.shape[1],  
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  
                  trainable=False, 
                  mask_zero=True)(inputs)                
                                                    

gru = GRU(units=100,                          
          return_sequences=False)(embed)      
gru = Dropout(0.2)(gru)
gru = BatchNormalization()(gru)
outputs = Dense(units=2, 
              activation='softmax')(gru)
model_gru1 = Model(inputs=inputs,
              outputs=outputs)



model_gru1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         40000200  
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                           

In [27]:
start = timeit.default_timer()

model_gru1.compile(
                optimizer = tf.keras.optimizers.RMSprop(learning_rate= 1e-3, name='RMSprop'),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])


history = model_gru1.fit(X_train, y_train,
                    batch_size= 100,
                    epochs= 10) 

stop = timeit.default_timer()
print('Time: ', stop - start) 

list_times1.append(stop-start)

Epoch 1/10
45/45 [==============================] - 7s 29ms/step - loss: 0.5053 - accuracy: 0.7614
Epoch 2/10
45/45 [==============================] - 1s 28ms/step - loss: 0.1858 - accuracy: 0.9493
Epoch 3/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0934 - accuracy: 0.9753
Epoch 4/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0763 - accuracy: 0.9794
Epoch 5/10
45/45 [==============================] - 1s 29ms/step - loss: 0.0602 - accuracy: 0.9827
Epoch 6/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0472 - accuracy: 0.9868
Epoch 7/10
45/45 [==============================] - 1s 27ms/step - loss: 0.0381 - accuracy: 0.9897
Epoch 8/10
45/45 [==============================] - 1s 29ms/step - loss: 0.0320 - accuracy: 0.9915
Epoch 9/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0285 - accuracy: 0.9917
Epoch 10/10
45/45 [==============================] - 1s 28ms/step - loss: 0.0221 - accuracy: 0.9928
Time:  25

In [45]:
performance1 = []

In [46]:

result = model_gru1.evaluate(X_valid,y_valid)
print("loss và accuracy trên tập val là {} {}".format(result[0],result[1]))

35/35 [==============================] - 0s 10ms/step - loss: 0.0818 - accuracy: 0.9749
loss và accuracy trên tập val là 0.08184133470058441 0.9748879075050354


In [47]:
performance1.append(result[1])

In [29]:

inputs = Input(shape=(None,))          
embed = Embedding(input_dim=embedding_matrix.shape[0],   
                  output_dim=embedding_matrix.shape[1],  
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  
                  mask_zero=True)(inputs)                
                                                    

gru = GRU(units=100,                         
            return_sequences=True,
            return_state=True)(embed)     
print(gru)
gru = GlobalAveragePooling1D()(gru[0])
gru = Dropout(0.2)(gru)
gru = BatchNormalization()(gru)
outputs = Dense(units=2, activation='softmax')(gru)
model_gru2 = Model(inputs=inputs,
              outputs=outputs)


model_gru2.summary()

[<KerasTensor: shape=(None, None, 100) dtype=float32 (created by layer 'gru_1')>, <KerasTensor: shape=(None, 100) dtype=float32 (created by layer 'gru_1')>]
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         40000200  
                                                                 
 gru_1 (GRU)                 [(None, None, 100),       60600     
                              (None, 100)]                       
                                                                 
 global_average_pooling1d_1   (None, 100)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_3 (Dropout)         (None

In [30]:
start = timeit.default_timer()
model_gru2.compile(
                optimizer = tf.keras.optimizers.RMSprop(learning_rate= 1e-3, name='RMSprop'),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

history = model_gru2.fit(X_train, y_train,
                    batch_size= 100,
                    epochs= 10) 


stop = timeit.default_timer()
print('Time: ', stop - start) 

list_times1.append(stop-start)

Epoch 1/10
45/45 [==============================] - 7s 40ms/step - loss: 0.4861 - accuracy: 0.7661
Epoch 2/10
45/45 [==============================] - 2s 38ms/step - loss: 0.1933 - accuracy: 0.9437
Epoch 3/10
45/45 [==============================] - 2s 37ms/step - loss: 0.0836 - accuracy: 0.9823
Epoch 4/10
45/45 [==============================] - 2s 36ms/step - loss: 0.0581 - accuracy: 0.9834
Epoch 5/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0375 - accuracy: 0.9901
Epoch 6/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0274 - accuracy: 0.9922
Epoch 7/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0225 - accuracy: 0.9930
Epoch 8/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0189 - accuracy: 0.9944
Epoch 9/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0146 - accuracy: 0.9951
Epoch 10/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0080 - accuracy: 0.9978
Time:  21

In [48]:

result = model_gru2.evaluate(X_valid,y_valid)
print("loss và accuracy trên tập val là {} {}".format(result[0],result[1]))

35/35 [==============================] - 0s 9ms/step - loss: 0.0724 - accuracy: 0.9812
loss và accuracy trên tập val là 0.07236023247241974 0.9811659455299377


In [49]:
performance1.append(result[1])


#### 2. So sánh giữa 2 cấu trúc của GRU và LSTM. Từ đó đưa ra kết luận của mình (có dẫn chứng )

**Công thức GRU và LSTM**

**LSTM Unit**
$$
\begin{align*}
\text{Forget gate} \\
f_{t} &= \sigma \left(W_{f}.\left[h_{t-1} , x_{t}\right]+b_{f}\right) \\
\text{Input gate} \\
i_{t} &= \sigma\left(W_{i}.\left[h_{t-1},x_{t} \right] + b_{i} \right) \\
\bar{C}_{t} &=\tanh\left(W_{c}.\left[h_{t-1}, x_{t}\right]+b_{C}\right) \\
C_{t} &=f_{t} * C_{t-1}+ i_{t}* \bar{C}_{t-1} \\
\text{Output gate} \\
o_{t} &=\sigma \left(W_{o}.\left[h_{t-1},x_{t}\right] + b_{o}\right) \\  
h_{t} &= o_{t} * \tanh\left(C_t\right)
\end{align*}
$$



**GRU Unit**
$$
\begin{align*}
\text{Update gate} \\
&z_{t}=\sigma\left(W_{z} \cdot\left[h_{t-1}, x_{t}\right]\right) \\
\text{Reset gate} \\
&r_{t}=\sigma\left(W_{r} \cdot\left[h_{t-1}, x_{t}\right]\right) \\
&\tilde{h}_{t}=\tanh \left(W \cdot\left[r_{t} * h_{t-1}, x_{t}\right]\right) \\
&h_{t}=\left(1-z_{t}\right) * h_{t-1}+z_{t} * \tilde{h}_{t}
\end{align*}
$$

Đối với **LSTM**, ta thấy rằng nó có tới 3 cổng là cổng đầu vào $i$, cổng quên $f$ và cổng đầu ra $o$ , trong khi **GRU** chỉ có 2 là cổng thiết lập lại $r$ và cổng cập nhật $z$. Nhìn vào kiến trúc của **LSTM** và **GRU** thì ta thấy rằng **LSTM** có phần phức tạp hơn do đó lượng parameters cần cho mô hình này sẽ nhiều hơn so với **GRU**.

Để so sánh giữa 2 mô hình này thì ta vẫn chưa đánh giá được bên nào nhỉnh hơn và cần phải chạy thử mới biết. Nhưng vẫn có 1 số trường hợp nếu ta muốn tìm ra kết quả nhanh hơn hay lượng dữ liệu hiện có bị ít thì **GRU** sẽ được ưu tiên chọn trước vì lượng parameters của nó ít hơn **LSTM**.

In [35]:
data_times = {
  "LSTM": list_times,
  "GRU": list_times1
}

print('Thời gian chạy của mô hình sử dụng LSTM và sử dụng GRU')
df = pd.DataFrame(data_times)
df

Thời gian chạy của mô hình sử dụng LSTM và sử dụng GRU


,LSTM,GRU
0,25.645567,25.379943
1,25.820474,21.865822


In [51]:
performances = {
  "LSTM": performance,
  "GRU": performance1
}

print('Performance của mô hình sử dụng LSTM và sử dụng GRU')
df = pd.DataFrame(performances)
df

Performance của mô hình sử dụng LSTM và sử dụng GRU


,LSTM,GRU
0,0.982063,0.974888
1,0.977578,0.981166


Với dữ liệu trên thì ta thấy rằng **LSTM** và **GRU** có performance tương đối ngang nhau và kết quả tốt nhất thuộc về **LSTM** nhưng xét về lượng thời gian chạy và số parameters thì **GRU** tốt hơn **LSTM** ở khía cạnh này. Vì vậy đối với data này , nếu ta muốn có kết quả tốt hơn thì chọn **LSTM**, còn nếu ta ưu tiên thời gian hơn thì sẽ chọn **GRU**.